# Test for Models module

## Librarie

In [1]:
import os, sys

sys.path.insert(0, os.pardir)

import torch
from torch import nn
import numpy as np

from src.models.unet3d import Unet3d
from constant import TILE_SIZE

## Model

### Try Half precision

In [2]:
try:
    c = nn.Conv3d(1, 32, 3).half()
    inputs = torch.randn(8, 1, 8, 256, 256).half()
    outputs = c(inputs)
    display(inputs.shape)
    display(outputs.shape)
except Exception as e:
    print(e)

"compute_columns3d" not implemented for 'Half'


### Try on GPU

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
try:
    c = nn.Conv3d(1, 32, 3).to(device=device)
    inputs = torch.randn(8, 1, 8, 256, 256).to(device=device)
    outputs = c(inputs)
    display(inputs.shape)
    display(outputs.shape)
except Exception as e:
    print(e)

Conv3D is not supported on MPS


### From 3D Images To 2D Mask

#### Max Pooling 3D

In [4]:
# pool of non-square window
m = nn.AdaptiveMaxPool3d((1, 256, 256))
inputs = torch.randn(8, 1, 64, 256, 256)
outputs = m(inputs)

display(inputs.shape)
display(outputs.shape)

torch.Size([8, 1, 64, 256, 256])

torch.Size([8, 1, 1, 256, 256])

#### Linear 

In [5]:
m = nn.Linear(20, 30)
input = torch.randn(128, 20)
output = m(input)
display(input.shape)
display(output.size())

torch.Size([128, 20])

torch.Size([128, 30])

In [6]:
input = torch.randn(32, 1, 5, 5)
display(input.shape)
# With default parameters
m = nn.Flatten()
output = m(input)
display(output.size())
# With non-default parameters
m = nn.Flatten(0, 2)
output = m(input)
display(output.size())

torch.Size([32, 1, 5, 5])

torch.Size([32, 25])

torch.Size([160, 5])

In [7]:
inputs = torch.randn(8, 1, 8, TILE_SIZE, TILE_SIZE)

### UNet3D

In [8]:
# UNet3D = Unet3d(1,TILE_SIZE)
# UNet3D(inputs)

# Pytorch-Lightning

## Metrics

In [9]:
from torchmetrics.classification import BinaryFBetaScore

threshold = .5
f05score = BinaryFBetaScore(0.5, threshold)
targets = torch.randint(0, 2, (8000,1))
predictions = torch.randint(0, 2, (8000,1))
f05score(targets, targets)

tensor(1.)

In [12]:
import os, sys

sys.path.insert(0, os.pardir)

from src.data.make_dataset import DatasetVesuvius
from src.models.metrics import F05Score
from constant import TRAIN_FRAGMENTS
from torch.utils.data import DataLoader
from src.utils import get_device
from src.utils import get_fragments_shape

device = torch.device('cpu')
print(device)
dataset = DatasetVesuvius(
        fragments=['1'],
        tile_size=TILE_SIZE,
        num_slices=2,
        random_slices=False,
        selection_thr=0.1,
        augmentation=False,
        device=device
    )
dataloader = DataLoader(dataset=dataset, batch_size=64, drop_last=True)
mask_sizes = get_fragments_shape(['1'], TILE_SIZE)
metric = F05Score(mask_sizes).to(device=device)
metric_noise = F05Score(mask_sizes).to(device=device)

cpu

Make image from ../data/raw/train/1


100%|██████████| 2/2 [00:00<00:00, 13.91it/s]


In [13]:
metric.device

device(type='mps', index=0)

In [11]:
import torch
import numpy as np

metric.reset()
metric_noise.reset()

for i, (fragment, bbox, mask, _) in enumerate(dataloader):
    # This metric have to give a score of 1
    mask = torch.squeeze(mask)
    metric.update(fragment, bbox, mask, mask)
    # print(len(fragment), bbox.shape, mask.shape)
    outputs = torch.tensor(np.zeros(mask.shape, np.int16))
    # print(outputs.shape)
    # # This metric have to give a score of ~0.5
    metric_noise.update(fragment, bbox, mask, outputs)
    # if i > 10:
    # break

print('Perfect F05Score:', metric.compute())
print('Noisy F05Score:', metric_noise.compute())

mps:0


RuntimeError: Placeholder storage has not been allocated on MPS device!

## Losses

In [ ]:
import os, sys

sys.path.insert(0, os.pardir)

from src.data.make_dataset import CustomDataset
from torch.utils.data import DataLoader
from src.utils import get_device

device = get_device()
from src.models.losses import CombinedLoss

dataset = CustomDataset(['1'], augmentation=False)
dataloader = DataLoader(dataset=dataset, batch_size=128)
criterion = CombinedLoss(bce_weight=1)

ImportError: cannot import name 'CustomDataset' from 'src.data.make_dataset' (/Users/titou/Documents/vesuvius-challenge-ink-detection/notebooks/../src/data/make_dataset.py)

In [ ]:
import torch
from torch.nn import BCELoss
import numpy as np

for i, (indexes, inputs, masks, coords) in enumerate(dataloader):
    outputs = torch.tensor(np.zeros(masks.shape, np.float32)).to(device=device)
    x = criterion(outputs, masks)
    print(x)
    break

tensor(29.4795, device='mps:0')
